In [0]:
%run "/Workspace/Dev/04. Common"

In [0]:
%skip
%sql

USE CATALOG `dev_catalog`;

In [0]:
%skip
checkpoint = spark.sql("""DESCRIBE EXTERNAL LOCATION checkpoints""").select("url").collect()[0]['url']
landing = spark.sql("""DESCRIBE EXTERNAL LOCATION landing""").select("url").collect()[0]['url']

In [0]:

dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")

In [0]:
def read_Traffic_Data():
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
    from pyspark.sql.functions import current_timestamp
    print("Reading the Raw Traffic Data :  ", end='')
    schema = StructType([
    StructField("Record_ID",IntegerType()),
    StructField("Count_point_id",IntegerType()),
    StructField("Direction_of_travel",StringType()),
    StructField("Year",IntegerType()),
    StructField("Count_date",StringType()),
    StructField("hour",IntegerType()),
    StructField("Region_id",IntegerType()),
    StructField("Region_name",StringType()),
    StructField("Local_authority_name",StringType()),
    StructField("Road_name",StringType()),
    StructField("Road_Category_ID",IntegerType()),
    StructField("Start_junction_road_name",StringType()),
    StructField("End_junction_road_name",StringType()),
    StructField("Latitude",DoubleType()),
    StructField("Longitude",DoubleType()),
    StructField("Link_length_km",DoubleType()),
    StructField("Pedal_cycles",IntegerType()),
    StructField("Two_wheeled_motor_vehicles",IntegerType()),
    StructField("Cars_and_taxis",IntegerType()),
    StructField("Buses_and_coaches",IntegerType()),
    StructField("LGV_Type",IntegerType()),
    StructField("HGV_Type",IntegerType()),
    StructField("EV_Car",IntegerType()),
    StructField("EV_Bike",IntegerType())
    ])
    rawTraffic_stream = (spark.readStream
                     .format("cloudFiles")
                     .option("cloudFiles.format", "csv")
                     .option("cloudFiles.schemaLocation",f"{checkpoint}rawTraffilcLoad/schemaInfer")
                     .option("header","true")
                     .schema(schema)
                     .load(landing+'raw_traffic/')
                     .withColumn("Extract_Time",current_timestamp())
                     )
    print('Reading Succcess !!')
    print('*******************')

    return rawTraffic_stream 

In [0]:
def write_Traffic_Data(read_Df,env):
    writeStream = (read_Df.writeStream
               .format("delta")
               .option("checkpointLocation",checkpoint+"rawTraffic/Checkpt")
               .outputMode("append")
               .trigger(availableNow=True)
               .queryName('rawTrafficWriteStream')
               .toTable(f"`{env}_catalog`.`bronze`.`raw_traffic`")
                )
    # writeStream.awaitTermination()
    print('Write Success')
    print("****************************")  


In [0]:
%skip
 spark.sql("""Select count(1) from `dev_catalog`.`bronze`.`raw_traffic`""").display()

In [0]:
def read_Road_Data():
    from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
    from pyspark.sql.functions import current_timestamp
    print("Reading the Raw Roads Data :  ", end='')
    schema = StructType([
        StructField('Road_ID',IntegerType()),
        StructField('Road_Category_Id',IntegerType()),
        StructField('Road_Category',StringType()),
        StructField('Region_ID',IntegerType()),
        StructField('Region_Name',StringType()),
        StructField('Total_Link_Length_Km',DoubleType()),
        StructField('Total_Link_Length_Miles',DoubleType()),
        StructField('All_Motor_Vehicles',DoubleType())
        ])
    
    rawRoads_stream = (spark.readStream
                     .format("cloudFiles")
                     .option("cloudFiles.format", "csv")
                     .option("cloudFiles.schemaLocation",f"{checkpoint}rawRoads/schemaInfer")
                     .option("header","true")
                     .schema(schema)
                     .load(landing+'raw_roads/')
                     )
    print('Reading Succcess !!')
    print('*******************')

    return rawRoads_stream

In [0]:

# display(streamingdf)

In [0]:
def write_roads_data(streamingdf,env):
    writeStream = (streamingdf.writeStream
               .format("delta")
               .option("checkpointLocation",checkpoint+"rawRoads/Checkpt")
               .outputMode("append")
               .trigger(availableNow=True)
               .queryName('rawRoadsWriteStream')
               .toTable(f"`{env}_catalog`.`bronze`.`raw_roads`")
    )
                
    # writeStream.awaitTermination()
    print('Write Success')
    print("****************************")  

In [0]:
#Read traffic data 
read_Df = read_Traffic_Data()

#Load traffic data
write_Traffic_Data(read_Df,env)

#Read traffic data 
streamingdf = read_Road_Data() 

#Load roads data
write_roads_data(streamingdf,env)

In [0]:
 %skip
# spark.sql("""Select * from `dev_catalog`.`bronze`.`raw_roads`""").display()
spark.sql("""Select count(1) from `dev_catalog`.`bronze`.`raw_traffic`""").display()

In [0]:
%skip
streamingf.printSchema()

In [0]:
%skip
spark.sql("""Select * from `dev_catalog`.`bronze`.`raw_roads`""").printSchema()